**Código de Escoragem**

Nesta etapa, é desenvolvido o código de escoragem, responsável por atribuir pontuações a novos dados que chegam após o modelo estar em produção. Esse código é o que efetivamente será implantado no ambiente de negócio e executado de acordo com a necessidade da operação, podendo rodar em tempo real (online scoring) ou em lotes (batch scoring).

O objetivo da escoragem é aplicar o modelo treinado sobre dados inéditos, classificando clientes quanto ao risco de churn e gerando as informações necessárias para suportar decisões estratégicas, como campanhas de retenção ou otimização de investimentos em marketing.

**1. Importar Bibliotecas**

In [2]:
# Bibliotecas necessárias para o projeto
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import itertools
import pickle
from sklearn.metrics import confusion_matrix, roc_curve, precision_recall_curve, roc_auc_score
from sklearn.model_selection import train_test_split


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


**2. Leitura dos dados a serem escorados**

Nesta etapa, são carregados os dados que passarão pelo processo de escoragem, ou seja, a aplicação do modelo preditivo já treinado para atribuir uma probabilidade de churn a cada cliente. Esses dados podem corresponder a uma nova base de clientes ou ao próprio conjunto de teste tratado. A leitura é feita a partir de um arquivo (como .csv) e resulta em um DataFrame preparado para receber o mesmo pipeline de pré-processamento utilizado na fase de treino, garantindo consistência e reprodutibilidade antes da aplicação do modelo.

Nessa fase, ajustamos a variável `TotalCharges` para numérica e removemos colunas que não participam da modelagem (`customerID`, `Churn` e índices auxiliares). O foco é preparar a base para aplicar exatamente o mesmo pipeline usado no treino

In [4]:
# Leitura dos dados a serem escorados
datapath = f'/content/drive/MyDrive/Colab Notebooks/Projetos/Predicao de Churn - Empresa Telecom/04 - Escoragem/WA_Fn-UseC_-Telco-Customer-Churn.csv'

df = pd.read_csv(datapath, sep=',')

if 'Churn' in df.columns:
    df['Churn'].replace({'Yes': 1, 'No': 0}, inplace=True)

df['TotalCharges'] = pd.to_numeric(df['TotalCharges'].replace(' ', pd.NA), errors='coerce')
df_escora = df.drop(columns=['Unnamed: 0','customerID','Churn'], errors='ignore')



/tmp/ipython-input-3100273074.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Churn'].replace({'Yes': 1, 'No': 0}, inplace=True)
/tmp/ipython-input-3100273074.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Churn'].replace({'Yes': 1, 'No': 0}, inplace=True)


**3. Carregar lista de variáveis que foram excluidas por excesso de nulos**


Nesta etapa, preenchemos valores ausentes das variáveis categóricas utilizando a **moda de cada coluna**, conforme definido no pré-processamento do treino. Isso garante consistência no tratamento de dados entre treino e escoragem.



In [5]:
with open('/content/drive/MyDrive/Colab Notebooks/Projetos/Predicao de Churn - Empresa Telecom/01 - Data Prep/prd_fillna.pkl', 'rb') as f:
    loaded_modes = pickle.load(f)

def fill_nulls_with_loaded_modes(dataframe, loaded_modes):
    categorical_columns_with_nulls = dataframe.select_dtypes(include='object').columns[
        dataframe.select_dtypes(include='object').isnull().any()
    ].tolist()
    for column in categorical_columns_with_nulls:
        if column in loaded_modes:
            dataframe[column].fillna(loaded_modes[column], inplace=True)
    return dataframe

df_escora = fill_nulls_with_loaded_modes(df_escora, loaded_modes)



**4. Label Encoder**

Variáveis categóricas de **alta cardinalidade** são transformadas em valores numéricos por meio dos encoders salvos. Isso evita explosão de dimensionalidade e garante que cada categoria receba o mesmo código atribuído na etapa de treino


In [6]:
with open('/content/drive/MyDrive/Colab Notebooks/Projetos/Predicao de Churn - Empresa Telecom/01 - Data Prep/prd_labelenc.pkl', 'rb') as f:
    loaded_data = pickle.load(f)

loaded_encoders = loaded_data['encoders']
loaded_columns = loaded_data['columns']

for col in loaded_columns:
    if col in loaded_encoders and col in df_escora.columns:
        df_escora[col] = loaded_encoders[col].transform(df_escora[col])


**5. OneHot Encoder**

Variáveis categóricas de **baixa cardinalidade** são convertidas em variáveis binárias (0/1) usando o encoder salvo. Essa transformação preserva a representatividade sem criar redundância, garantindo o mesmo padrão usado no treino.

In [7]:
# Carregar o encoder e a lista de colunas
with open('/content/drive/MyDrive/Colab Notebooks/Projetos/Predicao de Churn - Empresa Telecom/01 - Data Prep/prd_onehotenc.pkl', 'rb') as f:
    loaded_data = pickle.load(f)

loaded_encoder = loaded_data['encoder']
loaded_columns = loaded_data['columns']

encoded_data = loaded_encoder.transform(df_escora[loaded_columns])
encoded_cols = loaded_encoder.get_feature_names_out(loaded_columns)
encoded_df = pd.DataFrame(encoded_data, columns=encoded_cols, index=df_escora.index)

df_escora = pd.concat([df_escora.drop(loaded_columns, axis=1), encoded_df], axis=1)



**6. Padronização (Scaler)**

In [8]:
# Carregar o scaler
with open('/content/drive/MyDrive/Colab Notebooks/Projetos/Predicao de Churn - Empresa Telecom/01 - Data Prep/prd_scaler.pkl', 'rb') as f:
    loaded_scaler = pickle.load(f)

df_escora_scaled = loaded_scaler.transform(df_escora)
df_escora = pd.DataFrame(df_escora_scaled, columns=df_escora.columns, index=df_escora.index)


**7. Seleção de Features**

Mantemos apenas as variáveis que foram escolhidas na etapa de **Feature Selection**, garantindo que a escoragem seja feita exatamente com os atributos usados no treinamento do modelo campeão.


In [9]:
with open('/content/drive/MyDrive/Colab Notebooks/Projetos/Predicao de Churn - Empresa Telecom/02 - Feature Selection/prd_list_features.pkl', 'rb') as f:
    loaded_features = pickle.load(f)

abt_escora = df_escora[loaded_features]


loaded_features


['SeniorCitizen',
 'tenure',
 'MonthlyCharges',
 'TotalCharges',
 'gender_Male',
 'Partner_Yes',
 'Dependents_Yes',
 'MultipleLines_Yes',
 'InternetService_Fiber optic',
 'OnlineSecurity_Yes',
 'OnlineBackup_Yes',
 'TechSupport_Yes',
 'Contract_One year',
 'Contract_Two year',
 'PaperlessBilling_Yes',
 'PaymentMethod_Electronic check']

**8. Carregando modelo Campeão**

O modelo campeão, definido na etapa de modelagem (LightGBM), é carregado a partir do arquivo salvo em disco.


In [10]:
# Carregando modelo campeão
with open('/content/drive/MyDrive/Colab Notebooks/Projetos/Predicao de Churn - Empresa Telecom/03 - Modelagem/best_model_lightgbm.pkl', 'rb') as f:
    loaded_model = pickle.load(f)


loaded_model



LGBMClassifier(force_col_wise=True, max_depth=3, num_leaves=5, reg_alpha=0.1)

**9. Escoragem**


Aqui aplicamos o modelo nos dados tratados para gerar as **probabilidades de churn** (`score_1`) e a **classe prevista** (`class`). Em seguida, reintegramos o `customerID` e, se disponível, o target `Churn` apenas para fins de análise.



In [11]:

# Escorando base de treino

predprob = loaded_model.predict_proba(abt_escora)
predict = loaded_model.predict(abt_escora)

abt_escora['score_1'] = predprob[:,1]
abt_escora['class'] = predict

abt_escora = abt_escora.merge(df[['customerID','Churn']], left_index=True, right_index=True, how='left')
abt_escora.head()


/tmp/ipython-input-50816491.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  abt_escora['score_1'] = predprob[:,1]
/tmp/ipython-input-50816491.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  abt_escora['class'] = predict


SeniorCitizen    tenure  MonthlyCharges  TotalCharges  gender_Male  \
0      -0.436831 -1.284263       -1.159496     -0.996449    -1.032999   
1      -0.436831  0.063968       -0.260305     -0.175577     0.968056   
2      -0.436831 -1.243407       -0.363165     -0.961887     0.968056   
3      -0.436831  0.513379       -0.746399     -0.197096     0.968056   
4      -0.436831 -1.243407        0.195927     -0.942685    -1.032999   

   Partner_Yes  Dependents_Yes  MultipleLines_Yes  \
0     1.030066        -0.65845          -0.867153   
1    -0.970811        -0.65845          -0.867153   
2    -0.970811        -0.65845          -0.867153   
3    -0.970811        -0.65845          -0.867153   
4    -0.970811        -0.65845          -0.867153   

   InternetService_Fiber optic  OnlineSecurity_Yes  OnlineBackup_Yes  \
0                    -0.882690           -0.634162          1.370398   
1                    -0.882690            1.576884         -0.729715   
2                    -0.882690            1.576884          1.370398   
3                    -0.882690            1.576884         -0.729715   
4                     1.132901           -0.634162         -0.729715   

   TechSupport_Yes  Contract_One year  Contract_Two year  \
0        -0.638566          -0.521469          -0.562826   
1        -0.638566           1.917660          -0.562826   
2        -0.638566          -0.521469          -0.562826   
3         1.566008           1.917660          -0.562826   
4        -0.638566          -0.521469          -0.562826   

   PaperlessBilling_Yes  PaymentMethod_Electronic check   score_1  class  \
0              0.841222                        1.399708  0.678478      1   
1             -1.188746                       -0.714435  0.044716      0   
2              0.841222                       -0.714435  0.473796      0   
3             -1.188746                       -0.714435  0.050692      0   
4              0.841222                        1.399708  0.642230      1   

   customerID  Churn  
0  7590-VHVEG      0  
1  5575-GNVDE      0  
2  3668-QPYBK      1  
3  7795-CFOCW      0  
4  9237-HQITU      1

**10. Salvar Base Escorada**

Por fim, a base com os scores gerados é salva em arquivo `.csv`, ficando disponível para uso em relatórios, dashboards ou integração com sistemas de retenção.


In [12]:
abt_escora.to_csv('/content/drive/MyDrive/Colab Notebooks/Projetos/Predicao de Churn - Empresa Telecom/abt_scored_final.csv', index=False)


**Conclusão**

Ao final do projeto, o modelo desenvolvido mostrou-se capaz de prever a probabilidade de um cliente vir a cancelar o serviço, fornecendo informações valiosas para a adoção de estratégias preventivas e de retenção antes que o desligamento ocorra.

O desempenho obtido confirma sua eficácia na diferenciação entre clientes mais e menos propensos ao churn. Entre os algoritmos avaliados, o LightGBM apresentou os melhores resultados, destacando-se pela boa ordenação dos scores de probabilidade e pela robustez de sua métrica AUC, o que reforça a confiança em sua capacidade discriminativa.

Outro ponto de destaque foi a redução de falsos positivos, permitindo que os investimentos em ações de retenção sejam direcionados de forma mais eficiente, evitando custos desnecessários com clientes que não apresentam risco real de churn.

Além disso, a consistência observada nas métricas KS, Gini e AUC entre os conjuntos de treino e teste indica ausência de overfitting, garantindo que o modelo seja robusto e generalizável. Dessa forma, o LightGBM se consolida como uma ferramenta confiável para apoiar a tomada de decisões estratégicas voltadas à retenção de clientes.